In [ ]:
import json
import sys
import pandas as pd
from hccpy.hcc import HCCEngine
sys.path.append('../src')
import cb_utils

In [ ]:
query = """

WITH
    mems AS (
        SELECT
            m.id
          , DATE_PART('year', AGE(m.date_of_birth))                          age
          , UPPER(m.gender)                                                  sex
          , ROW_NUMBER() OVER (PARTITION BY m.id ORDER BY es.end_date DESC ) rn
          , CASE
--              "INS": Long Term Institutional
                WHEN ggroup = '1'                                                    THEN 'INS'
--              "CFD": Community Full Benefit Dual Disabled
                WHEN product = 'ecf' OR DATE_PART('year', AGE(m.date_of_birth)) < 65 THEN 'CFD'
--              "CFA": Community Full Benefit Dual Aged
                ELSE 'CFA' END AS                                            hcc_elg
          , product = 'ecf'                                                  is_disabled
          , es.line_of_business_id
          , ggroup
          , is_dual
          , MAX(cd.id) IS NOT NULL                                           is_esrd
        FROM
            cb.members m
            LEFT JOIN cb.claims c ON m.id = c.member_id AND m.mco_id = c.mco_id
            LEFT JOIN cb.claims_diagnosis cd ON c.id = cd.claim_id AND cd.diag = 'n18.6'
--             JOIN      cb.cb_risk_batch_members rm ON rm.member_id = m.id
            JOIN      cb.eligibility_segments es ON es.member_id = m.id
                AND start_date <= '2020-01-01'
        where m.mco_id in (1,2)
        GROUP BY m.id, DATE_PART('year', AGE(m.date_of_birth)), UPPER(m.gender), CASE
--              "INS": Long Term Institutional
                WHEN ggroup = '1'                                                    THEN 'INS'
--              "CFD": Community Full Benefit Dual Disabled
                WHEN product = 'ecf' OR DATE_PART('year', AGE(m.date_of_birth)) < 65 THEN 'CFD'
--              "CFA": Community Full Benefit Dual Aged
                ELSE 'CFA' END, product = 'ecf', es.line_of_business_id, ggroup, is_dual
        , es.end_date
    )
SELECT
    m.id
  , m.age
  , m.sex
  , m.hcc_elg
--              orec: str
--               Original reason for entitlement code.
--               - "0": Old age and survivor's insurance
--               - "1": Disability insurance benefits
--               - "2": End-stage renal disease
--               - "3": Both DIB and ESRD
  , CASE WHEN m.is_disabled AND m.is_esrd THEN '3' -- not perfect is_disabled flag
         WHEN m.is_esrd                   THEN '2'
         WHEN m.is_disabled OR m.age < 65 THEN '1'
         ELSE '0' END           orec
  , ARRAY_AGG(DISTINCT cd.diag) dx
FROM
    mems m
    JOIN cb.claims c ON c.member_id = m.id
    JOIN cb.claims_diagnosis cd ON c.id = cd.claim_id
WHERE
    DATE_PART('year', c.date_from) = 2020
    and m.rn = 1
GROUP BY
    1, 2, 3, 4, 5;

"""

In [ ]:
df = cb_utils.sql_query_to_df(query)
# pd.read_sql?

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.to_csv('all_tn_query_df.csv')
# df = pd.read_csv('all_tn_query_df.csv')

In [ ]:
results = []
he = HCCEngine(year='2020')
for i, r in df.iterrows():
    rp = he.profile(r.dx, age=r.age, sex=r.sex, elig=r.hcc_elg, orec=r.orec, medicaid=True)
    result = {}
    result['member_id'] = r.id
    result['raw'] = json.dumps(rp)
    result['risk_score'] = rp['risk_score']
    results.append(result)
# rp = he.profile()

In [ ]:
results_df = pd.DataFrame(results)

In [ ]:
results_df.head()

In [ ]:
results_df.shape

In [ ]:
results_df.describe()

In [ ]:
results_df.to_sql('raf_for_tn_mems_20210602', cb_utils.get_engine(), schema='junk', index=False, if_exists='replace')